<a href="https://colab.research.google.com/github/dmi3eva/araneae/blob/main/p3_araneae_wrapper/AraneaeWrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Preprocessing

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
%cd /content
%cd drive
%cd My\ Drive
%cd PhD
%cd Paper_01

/content
/content/drive
/content/drive/My Drive
/content/drive/My Drive/PhD
/content/drive/My Drive/PhD/Paper_01


In [4]:
%cd spider_preprocessing
from process_sql import *
%cd ..

/content/drive/My Drive/PhD/Paper_01/spider_preprocessing
/content/drive/My Drive/PhD/Paper_01


### Utils

In [5]:
from typing import Dict

In [6]:
def unfold_list():
  unfolded_list = []
  for _value in values:
    if isinstance(_value, list):
      unfolded_list += unfold_list(_value)
    else:
      new_value = _value
      if isinstance(_value, float) and (int(_value) - _value < 0.01):
        new_value = int(value)
      unfolded_list.append(new_value)
  return unfolded_list

In [7]:
def preprocess_SQL(sentence, splitter='select'):
    sentence = sentence.strip()
    tokens = sentence.split()

    # Убираем названия (новое)
    if ' as ' in sentence.lower():
      renaming = {}    
      for i, _token in enumerate(tokens):
        if _token.lower() == 'as':
          sentence = sentence.replace(f'AS {tokens[i + 1]} ', '')
          sentence = sentence.replace(tokens[i + 1], tokens[i - 1])
    

    # Сортируем аргументы SELECT
    tokens = sentence.split()
    select_part = []
    select_tokens = []
    before_tokens = []
    after_tokens = []
    select_flag = 'before'
    for _token in tokens:  
      if splitter in _token.lower():
        select_flag = 'during'
        before_tokens.append(_token)
      elif select_flag == 'during' and 'from' != _token.lower().strip():
        select_tokens.append(_token)
      else:
        if 'from' in _token.lower().strip():
          select_flag = 'after'
          after_tokens.append(_token)
        elif select_flag == 'after':
          after_tokens.append(_token)
        else:
          before_tokens.append(_token)

    select_part = ' '.join(select_tokens)
    select_tokens = select_part.split(',')
    select_tokens = sorted([_t.strip() for _t in select_tokens])
    select_part = ' , '.join(select_tokens)

    sentence = "{} {} {}".format(' '.join(before_tokens), select_part, ' '.join(after_tokens))

    # Убираем названия таблиц, если она одна
    if '.' in sentence and not 'join' in sentence.lower():
      tokens = sentence.split()          
      new_tokens = []
      for _token in tokens:  
        if '.' in _token:
          new_tokens.append(_token.split('.')[1])
        else:
          new_tokens.append(_token)
      sentence = ' '.join(new_tokens)
      
    result = sentence.replace('* ', '*').replace(' * ', '*').replace(' ,', ',').replace('( ', '(').replace(' )', ')')
    return result

In [8]:
DB_PATH = "datasets/spider/database"
SCHEMES_PATH = "datasets/spider/tables.json"

In [9]:
with open(SCHEMES_PATH) as table_file:
  schemes = json.load(table_file)

dbs = {_s['db_id']: _s for _s in schemes}

In [89]:
def extract_column_from_col_unit(scheme, col_unit):
  col_id = col_unit[1]
  column = scheme["column_names_original"][col_id]
  table_id = column[0] 
  return {
      "db_id": scheme["db_id"],
      "table": scheme["table_names_original"][table_id],
      "column": column[1]
  }

def extract_column_from_val_unit(scheme, val_unit):
  col_unit_1 = val_unit[1]
  col_unit_2 = val_unit[2]
  columns = [extract_column_from_col_unit(scheme, col_unit_1)]
  if col_unit_2: 
    columns += [extract_column_from_col_unit(scheme, col_unit_2)]
  return columns 

def extract_column_from_table_unit(scheme, table_unit):
  if isinstance(table_unit, dict) and 'select' in val:
    return get_units_from_spider_sql(scheme['db_id'], table_unit[1])
  if isinstance(table_unit[1], tuple) and len(table_unit[1]) == 3: 
    return extract_column_from_col_unit(scheme, table_unit[1])
  return []

def extract_column_from_cond_unit(scheme, table_unit):
  units = []
  units += extract_column_form_val_unit(scheme, table_unit[2])
  units += extract_column_form_val(scheme, table_unit[3])
  units += extract_column_form_val(scheme, table_unit[4])
  return units

def extract_column_from_val(scheme, val):
  if isinstance(val, dict) and 'select' in val:
    return get_units_from_spider_sql(scheme['db_id'], val)
  return []

def extract_column_from_condition(scheme, condition):
  units = []
  conditions_amount = len(condition) // 2
  for i in range(conditions_amount):
    units += extract_column_from_cond_unit(scheme, condition[2 * i])
  return units  

def deduplicate(list_of_dict):
  deduplicated = []
  hashes = []
  for _dict in list_of_dict:
    dict_hash = [f"{_k}:{_v}" for _k, _v in _dict.items()]
    if dict_hash not in hashes:
      deduplicated.append(_dict)
      hashes.append(dict_hash)
  return deduplicated


def get_units_from_spider_sql(db_id: str, sql: dict):
  units = []  
  scheme = dbs[db_id]
  # Parsing of SELECT
  for select_unit in sql['select'][1]:
    val_unit = select_unit[1]
    units += extract_column_from_val_unit(scheme, val_unit)
  from_table_units = sql['from']['table_units'] 
  from_condition = sql['from']['conds']
  for _table_unit in from_table_units:
    units += extract_column_from_table_unit(scheme, _table_unit)
  units += extract_column_from_condition(scheme, from_condition)
  units += extract_column_from_condition(scheme, sql['where'])  
  for _group in sql['groupBy']:
    units += extract_column_from_col_unit(scheme, _group)
  if isinstance(sql['orderBy'], tuple) and len(sql['orderBy']) >= 2:
    for _val in sql['orderBy'][1]:
      units += extract_column_from_val_unit(scheme, _val)
  units += extract_column_from_condition(scheme, sql['having']) 
  if sql['intersect']:
    units += get_units_from_spider_sql(db_id, sql['intersect'])
  if sql['except']:
    units += get_units_from_spider_sql(db_id, sql['except'])
  if sql['union']:
    units += get_units_from_spider_sql(db_id, sql['union'])
  return deduplicate(units)

In [55]:
def get_units_from_sample(sample: dict):
  db_id = sample['db_id']
  return get_units_from_spider_sql(db_id, sample['sql'])

In [56]:
with open('datasets/araneae/araneae.json') as table_file:
  araneae = json.load(table_file)

In [93]:
sample = araneae[29]
sample['question']

'What are the names of the stadiums without any concerts?'

In [83]:
sample['query']

"SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'"

In [84]:
dbs['concert_singer']["table_names_original"]

['stadium', 'singer', 'concert', 'singer_in_concert']

In [94]:
units = get_units_from_sample(sample)
units

[{'column': 'Name', 'db_id': 'concert_singer', 'table': 'stadium'}]

In [81]:
sample['sql']['select'][1]

[[0, [0, [0, 9, False], None]],
 [0, [0, [0, 10, False], None]],
 [0, [0, [0, 13, False], None]]]

### Main class

In [ ]:
import pandas as pd
from copy import deepcopy
from typing import Tuple

In [ ]:
with open('datasets/araneae/binary_values.json') as table_file:
  binary_classification = json.load(table_file)

In [ ]:
class AraneaeWrapper:
  def __init__(self, samples_path, tables_path, db_path):
    self.samples = []
    self.db_path = db_path
    with open(samples_path) as sample_file:
      self.samples = json.load(sample_file)
    with open(tables_path) as tables_file:
      self.tables = json.load(tables_file)
    for _sample in self.samples:
      _sample = self.augment_sample(_sample)

  def augment_sample(self, sample):
    if 'description' not in sample.keys():
      sample['description'] = {}
    if 'tags' not in sample.keys():
      sample['tags'] = []    
    if 'question_toks' not in sample.keys():
      sample['question_toks'] = tokenize(sample['question'])      
    
    if 'sql' not in sample.keys():
      db_id = sample['db_id']
      db = get_schema(f"{self.db_path}/{db_id}/{db_id}.sqlite")
      schema = Schema(db)
      sql_dict = get_sql(schema, sample['query'])
      sample['sql'] = sql_dict
      query_no_value = sample['query']
      for _value in unfold_list(sql_dict['where']):
        if str(_value) in query_no_value:
          query_no_value = query_no_value.replace(str(_value), 'value')
      sample['question_toks_no_value'] = tokenize(query_no_value)
      
    augmented_sample = deepcopy(sample)
    augmented_sample = self.add_nl_tag(augmented_sample)
    return augmented_sample

  def add_nl_tag(self, sample):
    SHORT_THRESHOLD = 13
    LONG_THRESHOLD = 17
    if 'NL-length' in sample['description']:
      return sample
    tag = None
    augmented_sample = deepcopy(sample)
    request_len = len(sample['question_toks'])
    if request_len <= SHORT_THRESHOLD:
      tag = "NL-short"
    elif request_len <= LONG_THRESHOLD:
      tag = "NL-avg"
    else:
      tag = "NL-long"
    augmented_sample['tags'].append(tag)
    augmented_sample['description']['NL-length'] = tag
    return augmented_sample

  def add_binary_tag(self, sample):
    db_id = sample["db_id"]
    
  def get_samples_with_tag(self, tag):
    samples_with_tag = [_s for _s in self.samples if tag in _s['tags']]
    return samples_with_tag

  def add_sample(self, nl, db_id, sql, source, tag=None, description: Tuple[str, str]=(None, None)):
    new_sample = {
        'db_id': db_id,
        'question': nl,
        'source': source,
        'query': preprocess_SQL(sql),
        'tags': [],
        'description': {}
    }   
    print(new_sample) 
    if tag:
      new_sample['tags'].append(tag)
    if description[0] and description[1]:
      new_sample['description'][description[0]] = description[1]
    new_sample = self.augment_sample(new_sample)
    self.samples.append(new_sample)

  def add_samples_from_csv(self, file_path, tag=None, description: Tuple[str, str]=(None, None)):
    new_samples = pd.read_csv(file_path)
    for ind, row in new_samples.iterrows():
      self.add_sample(row['nl'], row['db_id'], row['sql'], file_path, tag=tag, description=description)

  def save_in_json(self, file_path):
    with open(file_path, 'w') as json_file:
      json.dump(self.samples, json_file)

### Spider preprocessing

In [ ]:
%cd /content
%cd drive
%cd My\ Drive
%cd PhD
%cd Paper_01
%cd spider_preprocessing

/content
/content/drive
/content/drive/My Drive
/content/drive/My Drive/PhD
/content/drive/My Drive/PhD/Paper_01
/content/drive/My Drive/PhD/Paper_01/spider_preprocessing


In [ ]:
!git clone https://github.com/taoyds/spider

Cloning into 'spider'...
remote: Enumerating objects: 380, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 380 (delta 8), reused 0 (delta 0), pack-reused 361
Receiving objects: 100% (380/380), 44.95 MiB | 15.96 MiB/s, done.
Resolving deltas: 100% (102/102), done.
Checking out files: 100% (261/261), done.


### Тест

In [ ]:
%cd /content
%cd drive
%cd My\ Drive
%cd PhD
%cd Paper_01

/content
/content/drive
/content/drive/My Drive
/content/drive/My Drive/PhD
/content/drive/My Drive/PhD/Paper_01


In [ ]:
import json

In [ ]:
araneae = AraneaeWrapper("datasets/araneae/araneae.json", "datasets/spider/tables.json", "datasets/spider/database")

In [ ]:
araneae.add_samples_from_csv("datasets/to_add/binary_values.csv", tag='binary-values', description=('values', 'containing-binary'))

In [ ]:
araneae.get_samples_with_tag('binary-values')[0]

### Tests' zone

Tokenization from Spider

In [ ]:
tokens = tokenize("My dogs is good!")
tokens

['my', 'dogs', 'is', 'good', '!']

SQL-processing from Spider

In [ ]:
query = "SELECT name FROM singer"
schema_sql = get_schema("datasets/spider/database/singer/singer.sqlite")
schema = Schema(schema_sql)
sql = get_sql(schema, query)
sql

{'except': None,
 'from': {'conds': [], 'table_units': [('table_unit', '__singer__')]},
 'groupBy': [],
 'having': [],
 'intersect': None,
 'limit': None,
 'orderBy': [],
 'select': (False, [(0, (0, (0, '__singer.name__', False), None))]),
 'union': None,
 'where': []}

SQL-preprocessing from Araneae

In [ ]:
sentence = "SELECT T1.name , zaza, count(*), A FROM concert AS T1 GROUP BY t1.stadium_id"
preprocessed_sql = preprocess_SQL(sentence)
preprocessed_sql

'SELECT A, name, count(*), zaza FROM concert GROUP BY stadium_id'